In [3]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [4]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467).astype(np.float64)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

(467,)


In [5]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add).astype(np.float64)
        a = np.concatenate((a, adds))
    inputs.append(a)
inputs[0].shape

(53,)

In [6]:
use = np.stack(inputs).astype(np.float32)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation
use.shape

(467, 53)

In [7]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))
b.shape

(467, 53, 200)

In [8]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [9]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 53]), torch.Size([141, 53, 200]), torch.Size([141]))

# 0.6879432624113475 67轮

In [33]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 53, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=53,    # n_filters
                kernel_size=(1,201),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=(53,1))   # 在 2x2 空间里向下采样, output shape (16, 53, 1)
        )
#         self.out = nn.Linear(8480, 256)   # fully connected layer, output 10 classes
        self.output = nn.Linear(53, 7)   # fully connected layer, output 10 classes

    def forward(self, sen, x):
        sen = sen.reshape((sen.shape[0],1,sen.shape[1])) # (BATCH_SIZE,1,53)
        use = torch.bmm(sen.double(),x) # (BATCH_SIZE,1,200)
        use = torch.sigmoid(use) # (BATCH_SIZE,1,200)
        use = torch.softmax(use,2) # (BATCH_SIZE,1,200)
        use = use.reshape((use.shape[0],-1,1))
        use = torch.bmm(x,use)
        x = torch.cat((x,use),2)
        x = x.reshape((x.shape[0],1,x.shape[1],-1))
        x = self.conv1(x.double())
#         x = x.reshape(x.shape[0],x.shape[1],-1)
#         sen = sen.reshape((sen.shape[0],1,sen.shape[1]))
#         x = torch.cat((x,sen.double()),1)
#         x = x.reshape((x.shape[0],x.shape[1],x.shape[2],1))
#         x = self.maxpool(x)
        x = x.view(x.size(0), -1)   # 展平
        output = self.output(x.double())
        return output

In [35]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = CNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
start = time.time()
for epoch in range(1,301): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        torch.set_default_dtype(torch.double)
        pred = net(sen, batch_x.double())
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s , t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss} time: {time.time() - start}")

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 53, kernel_size=(1, 201), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=53, out_features=7, bias=True)
)
1: accuracy:0.2695035460992908 loss: 1.8569216676813904 time: 0.21128606796264648
2: accuracy:0.40425531914893614 loss: 1.6825805885609249 time: 0.42028069496154785
3: accuracy:0.524822695035461 loss: 1.3348510334047614 time: 0.6306407451629639
4: accuracy:0.5673758865248227 loss: 1.0152982538653457 time: 0.8642539978027344
5: accuracy:0.5815602836879432 loss: 0.7744289725664666 time: 1.1330580711364746
6: accuracy:0.6170212765957447 loss: 0.5916774058094098 time: 1.3383207321166992
7: accuracy:0.6524822695035462 loss: 0.3927998208724457 time: 1.5535848140716553
8: accuracy:0.6312056737588653 loss: 0.2420209302948902 time: 1.762558937072754
9: accuracy:0.624113475177305 loss: 0.1598346492561813 time: 1.977104902267456
10: 

96: accuracy:0.6879432624113475 loss: 0.00054747220472605 time: 21.43427085876465
97: accuracy:0.6879432624113475 loss: 0.0005953486840670002 time: 21.677482843399048
98: accuracy:0.6879432624113475 loss: 0.0005330939560191642 time: 21.89757800102234
99: accuracy:0.6879432624113475 loss: 0.0005995558921987459 time: 22.106060028076172
100: accuracy:0.6879432624113475 loss: 0.0005737499933822221 time: 22.314457893371582
101: accuracy:0.6879432624113475 loss: 0.0005431025661342633 time: 22.586254835128784
102: accuracy:0.6879432624113475 loss: 0.0005468784422710624 time: 22.793956756591797
103: accuracy:0.6879432624113475 loss: 0.0005449203312240668 time: 23.00202488899231
104: accuracy:0.6879432624113475 loss: 0.0005402685343339288 time: 23.29014301300049
105: accuracy:0.6879432624113475 loss: 0.0005319915355803165 time: 23.525513887405396
106: accuracy:0.6879432624113475 loss: 0.0005349829756870277 time: 23.742321968078613
107: accuracy:0.6879432624113475 loss: 0.0005480319638433302 tim

192: accuracy:0.6879432624113475 loss: 0.0002306925926394397 time: 42.49889588356018
193: accuracy:0.6879432624113475 loss: 0.0001642543347683529 time: 42.70416188240051
194: accuracy:0.6879432624113475 loss: 0.00018575276261474373 time: 42.95573973655701
195: accuracy:0.6879432624113475 loss: 0.00020266461195188917 time: 43.187559843063354
196: accuracy:0.6879432624113475 loss: 0.00017212599014295092 time: 43.390012979507446
197: accuracy:0.6879432624113475 loss: 0.00018630702974139015 time: 43.59045886993408
198: accuracy:0.6879432624113475 loss: 0.00021025028876033857 time: 43.789438009262085
199: accuracy:0.6879432624113475 loss: 0.00015874549797098174 time: 43.99490308761597
200: accuracy:0.6879432624113475 loss: 0.00018026706444263597 time: 44.19948697090149
201: accuracy:0.6879432624113475 loss: 0.00019364810513414606 time: 44.40020680427551
202: accuracy:0.6879432624113475 loss: 0.0001813422987651668 time: 44.60036873817444
203: accuracy:0.6879432624113475 loss: 0.0001651802905

288: accuracy:0.6879432624113475 loss: 9.770422985541569e-05 time: 62.601089000701904
289: accuracy:0.6879432624113475 loss: 0.00010565254081969907 time: 62.799152851104736
290: accuracy:0.6879432624113475 loss: 9.878719426226518e-05 time: 62.99910092353821
291: accuracy:0.6879432624113475 loss: 0.00010181135698026367 time: 63.19698095321655
292: accuracy:0.6879432624113475 loss: 0.00010676015214592383 time: 63.39206290245056
293: accuracy:0.6879432624113475 loss: 0.00010600478555483152 time: 63.587785959243774
294: accuracy:0.6879432624113475 loss: 8.953990845533357e-05 time: 63.78749179840088
295: accuracy:0.6879432624113475 loss: 9.220393231011822e-05 time: 63.9874050617218
296: accuracy:0.6879432624113475 loss: 9.295801770551496e-05 time: 64.1849467754364
297: accuracy:0.6879432624113475 loss: 8.087584969154499e-05 time: 64.38666701316833
298: accuracy:0.6879432624113475 loss: 7.981540654078095e-05 time: 64.59302496910095
299: accuracy:0.6879432624113475 loss: 9.60697712971026e-05 